# 0. Mount Gdrive

In [36]:
# mount drive https://datascience.stackexchange.com/questions/29480/uploading-images-folder-from-my-system-into-google-colab
# login with your google account and type authorization code to mount on your google drive.
import os
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


# 1. Import library

In [37]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
from os import listdir
import cv2
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image  as mpimg
from PIL import Image
from google.colab.patches import cv2_imshow
import imutils
import math

# 2. Define face pre-processing model

In [38]:
# multiple cascades: https://github.com/Itseez/opencv/tree/master/data/haarcascades
def extract_mask(SOURCE):
    img=cv2.imread(SOURCE)
    if img is None:
        return []

    # load pre-trained Haar cascade model for detect eye point
    face_cascade = cv2.CascadeClassifier('/gdrive/My Drive/Code/face_preprocessing/haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier('/gdrive/My Drive/Code/face_preprocessing/haarcascade_eye.xml')

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 3)

    # Append each eye point in the detected face
    eye_point = []
    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        
        eyes = eye_cascade.detectMultiScale(roi_gray)
        eyes = eyes[:2]
        for (ex,ey,ew,eh) in eyes:
            xx = ex + ew // 2
            yy = ey + eh // 2
            eye_point.append((x+xx, y+yy))
            r = (ew + eh) // 4

    # Return empty list if there are no detected eyes
    if len(eye_point) < 2:
        return []

    # Set the left and right eyes by comparing the location
    eye1 = eye_point[0]
    eye2 = eye_point[1]
    length = math.sqrt(((eye1[0] - eye2[0]) ** 2) + ((eye1[1] - eye2[1]) ** 2))

    if eye1[0] > eye2[0]:
        right = eye1
        left = eye2
    elif eye1[0] < eye2[0]:
        right = eye2
        left = eye1

    # Rotated the image to fit the eye level
    if left[1] > right[1]:
        angle = math.atan2((right[1]-left[1]), (right[0]-left[0]))
        theta = angle*(180/math.pi);
        rotated = imutils.rotate(img,theta)
    elif left[1] <= right[1]:
        angle = math.atan2((left[1]-right[1]), (right[0]-left[0]))
        theta = angle*(180/math.pi);
        rotated = imutils.rotate(img,-theta)

    # Crop the rotated image by estimated size face
    cropped = rotated[int(left[1] - 0.6 * length):int(left[1] + 1.8 * length), int(left[0] - 0.6 * length):int(right[0] + 0.6 * length)]
    
    if cropped.size == 0:
        return []
    
    # Resize the cropped image by mask region
    resized = cv2.resize(cropped, dsize = (120, 140), interpolation=cv2.INTER_AREA)
    mask_region = resized[50:, :]

    return mask_region

In [63]:
def process_data(SOURCE, PROCESS):
    dataset=[]

    for unitData in os.listdir(SOURCE):
        data=SOURCE+unitData
        if os.path.getsize(data)>0:
            dataset.append(unitData)
        else:
            print('Skipped '+unitData)
            print('Invalid file i.e zero size')
    
    for unitData in dataset:
        temp_set=SOURCE+unitData
        final_set=PROCESS+unitData # directory
        process_img=extract_mask(temp_set) # img
        if process_img==[]:
            pass
        else:
            cv2.imwrite(final_set, process_img)

DATA_DIR = "/gdrive/My Drive/dataset_new/"
PROCESS_DIR = DATA_DIR + "process/"

YES_SOURCE_DIR = DATA_DIR + "with_mask/"
YES_PROCESS_DIR = PROCESS_DIR + "with_mask/"
INCOR_SOURCE_DIR = DATA_DIR + "incorrect_mask/"
INCOR_PROCESS_DIR = PROCESS_DIR + "incorrect_mask/"
NO_SOURCE_DIR = DATA_DIR + "without_mask/"
NO_PROCESS_DIR= PROCESS_DIR + "without_mask/"

In [40]:
print("The number of images with facemask labelled 'yes':",len(os.listdir(YES_SOURCE_DIR)))
print("The number of images with facemask labelled 'no':",len(os.listdir(NO_SOURCE_DIR)))
print("The number of images with facemask labelled 'incorrect':",len(os.listdir(INCOR_SOURCE_DIR)))

The number of images with facemask labelled 'yes': 1916
The number of images with facemask labelled 'no': 1919
The number of images with facemask labelled 'incorrect': 1987


In [41]:
process_data(YES_SOURCE_DIR, YES_PROCESS_DIR)
process_data(NO_SOURCE_DIR, NO_PROCESS_DIR)
process_data(INCOR_SOURCE_DIR, INCOR_PROCESS_DIR)

print("The number of images with facemask labelled 'yes':",len(os.listdir(YES_PROCESS_DIR)))
print("The number of images with facemask labelled 'no':",len(os.listdir(NO_PROCESS_DIR)))
print("The number of images with facemask labelled 'incorrect':",len(os.listdir(INCOR_PROCESS_DIR)))

The number of images with facemask labelled 'yes': 85
The number of images with facemask labelled 'no': 579
The number of images with facemask labelled 'incorrect': 764


#3. Split Dataset into Training, Validation, Testing Dataset

In [108]:
def split_data(SOURCE, TRAINING, VALIDATION, SPLIT_SIZE):
    dataset = []
    
    for unitData in os.listdir(SOURCE):
        data = SOURCE + unitData
        if(os.path.getsize(data) > 0):
            dataset.append(unitData)
        else:
            print('Skipped ' + unitData)
            print('Invalid file i.e zero size')
    
    train_set_length = int(len(dataset) * SPLIT_SIZE)
    test_set_length = int(len(dataset) - train_set_length)
    train_set = dataset[0:train_set_length]
    test_set = dataset[-test_set_length:]
       
    for unitData in train_set:
        temp_train_set = SOURCE + unitData
        final_train_set = TRAINING + unitData
        copyfile(temp_train_set, final_train_set)
    
    for unitData in test_set:
        temp_valid_set = SOURCE + unitData
        final_valid_set = VALIDATION + unitData
        copyfile(temp_valid_set, final_valid_set)

TRAINING_DIR = DATA_DIR + "training/"
VALIDATION_DIR = DATA_DIR + "validation/"
TESTING_DIR = DATA_DIR + "testing/"

TRAINING_YES_DIR = TRAINING_DIR + "with_mask/"
VALIDATION_YES_DIR = VALIDATION_DIR + "with_mask/"
TESTING_YES_DIR = TESTING_DIR + "with_mask/"

TRAINING_INCOR_DIR = TRAINING_DIR + "incorrect_mask/"
VALIDATION_INCOR_DIR = VALIDATION_DIR + "incorrect_mask/"
TESTING_INCOR_DIR = TESTING_DIR + "incorrect_mask/"

TRAINING_NO_DIR = TRAINING_DIR + "without_mask/"
VALIDATION_NO_DIR = VALIDATION_DIR + "without_mask/"
TESTING_NO_DIR = TESTING_DIR + "without_mask/"
split_size = .8

# split datasets into training set and testing set
split_data(YES_PROCESS_DIR, TRAINING_YES_DIR, VALIDATION_YES_DIR, split_size)
split_data(INCOR_PROCESS_DIR, TRAINING_INCOR_DIR, VALIDATION_INCOR_DIR, split_size)
split_data(NO_PROCESS_DIR, TRAINING_NO_DIR, VALIDATION_NO_DIR, split_size)

In [91]:
print("The number of images with facemask in the training set labelled 'with_mask':", len(os.listdir(TRAINING_YES_DIR)))
print("The number of images with facemask in the validation set labelled 'with_mask':", len(os.listdir(VALIDATION_YES_DIR)))
print("The number of images with facemask in the test set labelled 'with_mask':", len(os.listdir(TESTING_YES_DIR)))

print("The number of images without facemask in the training set labelled 'incorrect_mask':", len(os.listdir(TRAINING_INCOR_DIR)))
print("The number of images with facemask in the validation set labelled 'incorrect_mask':", len(os.listdir(VALIDATION_INCOR_DIR)))
print("The number of images without facemask in the test set labelled 'incorrect_mask':", len(os.listdir(TESTING_INCOR_DIR)))

print("The number of images without facemask in the training set labelled 'without_mask':", len(os.listdir(TRAINING_NO_DIR)))
print("The number of images with facemask in the validation set labelled 'without_mask':", len(os.listdir(VALIDATION_NO_DIR)))
print("The number of images without facemask in the test set labelled 'without_mask':", len(os.listdir(TESTING_NO_DIR)))

The number of images with facemask in the training set labelled 'with_mask': 68
The number of images with facemask in the validation set labelled 'with_mask': 17
The number of images with facemask in the test set labelled 'with_mask': 0
The number of images without facemask in the training set labelled 'incorrect_mask': 0
The number of images with facemask in the validation set labelled 'incorrect_mask': 0
The number of images without facemask in the test set labelled 'incorrect_mask': 153
The number of images without facemask in the training set labelled 'without_mask': 0
The number of images with facemask in the validation set labelled 'without_mask': 0
The number of images without facemask in the test set labelled 'without_mask': 116


#4. Make CNN Model

##4.1. One CNN model with 3 Classes

In [46]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (5, 5), activation='relu', input_shape=(90, 120, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (5, 5), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    # 2차원 array pixel을 1차원 array로 변환
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 128개의 node 층
    tf.keras.layers.Dense(128, activation='relu'),
    # 출력층은 한 개의 node로 구성되고 0-1 사의의 값을 출력
    tf.keras.layers.Dense(3, activation='softmax')
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 86, 116, 32)       2432      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 43, 58, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 39, 54, 128)       102528    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 19, 27, 128)       0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 65664)             0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 65664)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)              

In [47]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              # metrics: training 단계와 test 단계를 모니터링 하기 위해 사용, 해당 코드에서는 올바른 이미지의 비율인 accuracy를 사용
              metrics=['accuracy'])

In [109]:
# 값을 0-1 사이로 조정
train_datagen=ImageDataGenerator(rescale=1.0/255)
train_generator=train_datagen.flow_from_directory(TRAINING_DIR, 
                                                  batch_size=10, 
                                                  target_size=(90, 120))

validation_datagen=ImageDataGenerator(rescale=1.0/255)
validation_generator=validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                  batch_size=10, 
                                                  target_size=(90, 120))

testing_datagen=ImageDataGenerator(rescale=1.0/255)
testing_generator=testing_datagen.flow_from_directory(TESTING_DIR, 
                                                  batch_size=10, 
                                                  target_size=(90, 120))

Found 1142 images belonging to 3 classes.
Found 286 images belonging to 3 classes.
Found 286 images belonging to 3 classes.


In [49]:
hitory=model.fit_generator(train_generator,
                           epochs=30, 
                           validation_data=validation_generator)

Epoch 1/30
115/115 [==============================] - 3s 29ms/step - loss: 0.2752 - accuracy: 0.8765 - val_loss: 0.1287 - val_accuracy: 0.9406
Epoch 2/30
115/115 [==============================] - 3s 22ms/step - loss: 0.0918 - accuracy: 0.9588 - val_loss: 0.0878 - val_accuracy: 0.9650
Epoch 3/30
115/115 [==============================] - 2s 21ms/step - loss: 0.0814 - accuracy: 0.9641 - val_loss: 0.1129 - val_accuracy: 0.9720
Epoch 4/30
115/115 [==============================] - 2s 22ms/step - loss: 0.0771 - accuracy: 0.9597 - val_loss: 0.0819 - val_accuracy: 0.9685
Epoch 5/30
115/115 [==============================] - 2s 21ms/step - loss: 0.0725 - accuracy: 0.9650 - val_loss: 0.0784 - val_accuracy: 0.9790
Epoch 6/30
115/115 [==============================] - 3s 22ms/step - loss: 0.0476 - accuracy: 0.9772 - val_loss: 0.0797 - val_accuracy: 0.9755
Epoch 7/30
115/115 [==============================] - 2s 20ms/step - loss: 0.0331 - accuracy: 0.9825 - val_loss: 0.1036 - val_accuracy: 0.9825

In [ ]:
model.save('/gdrive/My Drive/Code/face_mask_detection_CNN/saved_incor_model')

In [110]:
new_model=tf.keras.models.load_model('/gdrive/My Drive/Code/face_mask_detection_CNN/saved_incor_model')

new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 86, 116, 32)       2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 43, 58, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 39, 54, 128)       102528    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 19, 27, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 65664)             0         
_________________________________________________________________
dropout (Dropout)            (None, 65664)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               8

In [111]:
results=new_model.evaluate(testing_generator)
for name, value in zip(new_model.metrics_names, results):
    print("%s: %.3f" %(name, value))

29/29 [==============================] - 0s 16ms/step - loss: 0.0901 - accuracy: 0.9720
loss: 0.090
accuracy: 0.972


##4.2. Two CNN Models with 2 Classes

###4.2.1. Classifying With mask and Without mask

In [51]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (5, 5), activation='relu', input_shape=(90, 120, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (5, 5), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    # 2차원 array pixel을 1차원 array로 변환
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 128개의 node 층
    tf.keras.layers.Dense(128, activation='relu'),
    # 출력층은 한 개의 node로 구성되고 0-1 사의의 값을 출력
    tf.keras.layers.Dense(2, activation='softmax')
])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 86, 116, 32)       2432      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 43, 58, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 39, 54, 128)       102528    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 19, 27, 128)       0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 65664)             0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 65664)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)              

In [52]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              # metrics: training 단계와 test 단계를 모니터링 하기 위해 사용, 해당 코드에서는 올바른 이미지의 비율인 accuracy를 사용
              metrics=['accuracy'])

In [100]:
TRAINING_DIR = DATA_DIR + "training2/"
VALIDATION_DIR = DATA_DIR + "validation2/"
TESTING_DIR = DATA_DIR + "testing2/"

TRAINING_YES_DIR = TRAINING_DIR + "with_mask/"
VALIDATION_YES_DIR = VALIDATION_DIR + "with_mask/"
TESTING_YES_DIR = TESTING_DIR + "with_mask/"

TRAINING_NO_DIR = TRAINING_DIR + "without_mask/"
VALIDATION_NO_DIR = VALIDATION_DIR + "without_mask/"
TESTING_NO_DIR = TESTING_DIR + "without_mask/"
split_size = .8

# split datasets into training set and testing set
split_data(YES_PROCESS_DIR, TRAINING_YES_DIR, VALIDATION_YES_DIR, split_size)
split_data(NO_PROCESS_DIR, TRAINING_NO_DIR, VALIDATION_NO_DIR, split_size)

In [101]:
# 값을 0-1 사이로 조정
train_datagen=ImageDataGenerator(rescale=1.0/255)
train_generator=train_datagen.flow_from_directory(TRAINING_DIR, 
                                                  batch_size=10, 
                                                  target_size=(90, 120))

validation_datagen=ImageDataGenerator(rescale=1.0/255)
validation_generator=validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                  batch_size=10, 
                                                  target_size=(90, 120))

testing_datagen=ImageDataGenerator(rescale=1.0/255)
testing_generator=testing_datagen.flow_from_directory(TESTING_DIR, 
                                                  batch_size=10, 
                                                  target_size=(90, 120))

Found 531 images belonging to 2 classes.
Found 133 images belonging to 2 classes.
Found 112 images belonging to 2 classes.


In [57]:
hitory=model.fit_generator(train_generator,
                           epochs=30, 
                           validation_data=validation_generator)

Epoch 1/30
54/54 [==============================] - 1s 25ms/step - loss: 0.4479 - accuracy: 0.8418 - val_loss: 0.3907 - val_accuracy: 0.9023
Epoch 2/30
54/54 [==============================] - 1s 26ms/step - loss: 0.1434 - accuracy: 0.9492 - val_loss: 0.1879 - val_accuracy: 0.9699
Epoch 3/30
54/54 [==============================] - 1s 21ms/step - loss: 0.1324 - accuracy: 0.9567 - val_loss: 0.1227 - val_accuracy: 0.9549
Epoch 4/30
54/54 [==============================] - 1s 21ms/step - loss: 0.1180 - accuracy: 0.9642 - val_loss: 0.1687 - val_accuracy: 0.9474
Epoch 5/30
54/54 [==============================] - 1s 21ms/step - loss: 0.1447 - accuracy: 0.9623 - val_loss: 0.1704 - val_accuracy: 0.9624
Epoch 6/30
54/54 [==============================] - 1s 21ms/step - loss: 0.1183 - accuracy: 0.9661 - val_loss: 0.1414 - val_accuracy: 0.9624
Epoch 7/30
54/54 [==============================] - 1s 21ms/step - loss: 0.1431 - accuracy: 0.9586 - val_loss: 0.3338 - val_accuracy: 0.9098
Epoch 8/30
54

In [58]:
model.save('/gdrive/My Drive/Code/face_mask_detection_CNN/saved_new_model')

In [102]:
new_model=tf.keras.models.load_model('/gdrive/My Drive/Code/face_mask_detection_CNN/saved_new_model')

new_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 86, 116, 32)       2432      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 43, 58, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 39, 54, 128)       102528    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 19, 27, 128)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 65664)             0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 65664)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)              

In [103]:
results=new_model.evaluate(testing_generator)
for name, value in zip(new_model.metrics_names, results):
    print("%s: %.3f" %(name, value))

12/12 [==============================] - 21s 2s/step - loss: 0.0402 - accuracy: 0.9911
loss: 0.040
accuracy: 0.991


###4.2.2 Classifying Incorrect mask and Without mask

In [82]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (5, 5), activation='relu', input_shape=(90, 120, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (5, 5), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    # 2차원 array pixel을 1차원 array로 변환
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 128개의 node 층
    tf.keras.layers.Dense(128, activation='relu'),
    # 출력층은 한 개의 node로 구성되고 0-1 사의의 값을 출력
    tf.keras.layers.Dense(2, activation='softmax')
])

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 86, 116, 32)       2432      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 43, 58, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 39, 54, 128)       102528    
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 19, 27, 128)       0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 65664)             0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 65664)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)              

In [83]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              # metrics: training 단계와 test 단계를 모니터링 하기 위해 사용, 해당 코드에서는 올바른 이미지의 비율인 accuracy를 사용
              metrics=['accuracy'])

In [104]:
TRAINING_DIR = DATA_DIR + "training3/"
VALIDATION_DIR = DATA_DIR + "validation3/"
TESTING_DIR = DATA_DIR + "testing3/"

TRAINING_INCOR_DIR = TRAINING_DIR + "incorrect_mask/"
VALIDATION_INCOR_DIR = VALIDATION_DIR + "incorrect_mask/"
TESTING_INCOR_DIR = TESTING_DIR + "incorrect_mask/"

TRAINING_NO_DIR = TRAINING_DIR + "without_mask/"
VALIDATION_NO_DIR = VALIDATION_DIR + "without_mask/"
TESTING_NO_DIR = TESTING_DIR + "without_mask/"
split_size = .8

# split datasets into training set and testing set
split_data(INCOR_PROCESS_DIR, TRAINING_INCOR_DIR, VALIDATION_INCOR_DIR, split_size)
split_data(NO_PROCESS_DIR, TRAINING_NO_DIR, VALIDATION_NO_DIR, split_size)

In [105]:
# 값을 0-1 사이로 조정
train_datagen=ImageDataGenerator(rescale=1.0/255)
train_generator=train_datagen.flow_from_directory(TRAINING_DIR, 
                                                  batch_size=10, 
                                                  target_size=(90, 120))

validation_datagen=ImageDataGenerator(rescale=1.0/255)
validation_generator=validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                  batch_size=10, 
                                                  target_size=(90, 120))

testing_datagen=ImageDataGenerator(rescale=1.0/255)
testing_generator=testing_datagen.flow_from_directory(TESTING_DIR, 
                                                  batch_size=10, 
                                                  target_size=(90, 120))

Found 1074 images belonging to 2 classes.
Found 269 images belonging to 2 classes.
Found 269 images belonging to 2 classes.


In [88]:
hitory=model.fit_generator(train_generator,
                           epochs=30, 
                           validation_data=validation_generator)

Epoch 1/30
108/108 [==============================] - 3s 25ms/step - loss: 0.1981 - accuracy: 0.9534 - val_loss: 0.0682 - val_accuracy: 0.9777
Epoch 2/30
108/108 [==============================] - 2s 22ms/step - loss: 0.0532 - accuracy: 0.9907 - val_loss: 0.0744 - val_accuracy: 0.9926
Epoch 3/30
108/108 [==============================] - 2s 23ms/step - loss: 0.0560 - accuracy: 0.9851 - val_loss: 0.0591 - val_accuracy: 0.9814
Epoch 4/30
108/108 [==============================] - 2s 21ms/step - loss: 0.5322 - accuracy: 0.9544 - val_loss: 4.9493 - val_accuracy: 0.6654
Epoch 5/30
108/108 [==============================] - 2s 23ms/step - loss: 0.5574 - accuracy: 0.9507 - val_loss: 0.1053 - val_accuracy: 0.9814
Epoch 6/30
108/108 [==============================] - 2s 23ms/step - loss: 0.0452 - accuracy: 0.9888 - val_loss: 0.0842 - val_accuracy: 0.9926
Epoch 7/30
108/108 [==============================] - 2s 23ms/step - loss: 0.3176 - accuracy: 0.9758 - val_loss: 0.3845 - val_accuracy: 0.9740

In [89]:
model.save('/gdrive/My Drive/Code/face_mask_detection_CNN/class2_model')

In [106]:
new_model=tf.keras.models.load_model('/gdrive/My Drive/Code/face_mask_detection_CNN/class2_model')

new_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 86, 116, 32)       2432      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 43, 58, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 39, 54, 128)       102528    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 19, 27, 128)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 65664)             0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 65664)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)              

In [107]:
results=new_model.evaluate(testing_generator)
for name, value in zip(new_model.metrics_names, results):
    print("%s: %.3f" %(name, value))

27/27 [==============================] - 0s 15ms/step - loss: 0.1039 - accuracy: 0.9888
loss: 0.104
accuracy: 0.989


#5. Video to Frame

In [ ]:
def video2frame(invideofilename, save_path):
    vidcap = cv2.VideoCapture(invideofilename)
    count = 0
    while True:
      success,image = vidcap.read()
      if not success:
          break
      fname = "{}.jpg".format("{0:05d}".format(count))
      cv2.imwrite(save_path + fname, image) # save frame as JPEG file
      count += 1
    print("{} images are extracted in {}.". format(count, save_path))

In [ ]:
invideofilename = '/gdrive/My Drive/jisu_mp4.mp4'
save_path = '/gdrive/My Drive/mp4_capture/'
video2frame(invideofilename,save_path)

In [ ]:
for i in range(509):
    img_string = '/gdrive/My Drive/mp4_capture/{0:05d}.jpg'.format(i)
    img=extract_mask(img_string)
    img=(np.expand_dims(img, 0))

    # predictions_single=model.predict(img)
    # np.argmax(predictions_single[0])
    print("{0:05d}:".format(i), end = " ")
    try:
        predictions_single=new_model.predict(img)
    except:
        print("")
        continue
    print(np.argmax(predictions_single[0]))